In [ ]:

!pip install bitsandbytes-cuda113

In [14]:
!pip install wandb

In [3]:
!pip install pytorch-lightning

  Using cached pytorch_lightning-1.6.0-py3-none-any.whl (582 kB)
  Using cached pyDeprecate-0.3.2-py3-none-any.whl (10 kB)
  Using cached torchmetrics-0.7.3-py3-none-any.whl (398 kB)
Note: you may need to restart the kernel to use updated packages.


In [17]:
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")

device = 'cuda:1'
model = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B").to(device)#results/checkpoint-45000


In [67]:
enc = tokenizer.encode_plus('how to poop',return_tensors='pt').to(device)

In [68]:

input_ids = enc['input_ids']
attention_mask = enc['attention_mask']
#model(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:1! (when checking arugment for argument index in method wrapper_index_select)

In [1]:
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
import pytorch_lightning as pl
tokenizer = GPT2Tokenizer.from_pretrained("EleutherAI/gpt-neo-2.7B")
df = pd.read_csv("../data_kaggle/sudoku.csv")

X = df.quizzes.values
y = df.solutions.values

prompt = ["0","1","2","3","4","5","6","7","8","9"]
map_values = list(tokenizer(prompt, return_tensors="pt").input_ids.numpy().flatten())
map_value_to_token = dict(zip(prompt, map_values))
map_token_to_value = {map_value_to_token[x]:x for x in map_value_to_token}
bos = tokenizer.encode(tokenizer.bos_token)[0]
map_token_to_value[bos] = ''

# [bos] + [:-1]
X = [list(map(map_value_to_token.__getitem__, x)) for x in X]
y = [list(map(map_value_to_token.__getitem__, x)) for x in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42, shuffle=True)
    



In [2]:
from torch.utils.data import Dataset
import torch

class SudokuDataset(Dataset):
    
    def __init__(self, X, Y):
        
        self.exampls = []
        self.input_ids = []
        self.attn_masks = []
        self.labels = []
        
        #tt = torch.zeros((1, 4), dtype=torch.int)
        for x, y in zip(X, Y):
            self.input_ids.append(torch.tensor(x).reshape((1, 81)))
            
            self.labels.append(torch.tensor(y).reshape((1, 81)))
            
            self.attn_masks.append(torch.ones((1, 81), dtype=torch.int64))
            
            self.exampls.append(torch.cat((torch.tensor(x),torch.tensor(y)),0))
    
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, idx):
        return self.input_ids[idx], self.attn_masks[idx], self.labels[idx]
    

In [3]:
dataset = SudokuDataset(X_train,y_train)

In [4]:
class Args:
    def __init__(self):
        self.overwrite_cache = False
        #self.tokenizer_name = "EleutherAI/gpt-neo-2.7B"
        #self.model_name_or_path = 'EleutherAI/gpt-neo-2.7B'
        self.output_dir = "sudoku_freezed_2.7"
        self.deepspeed=False
        if not self.deepspeed:
          self.fp16 = True
          self.fp16_opt_level = "O1"
          self.n_gpu = 0
          self.local_rank = 1
          self.device = torch.device("cuda")
        
        self.weight_decay = 0.01
        self.eval_batch_size = 2
        self.train_batch_size = 2
        self.gradient_accumulation_steps = 5
        self.lr = 1e-5
        self.warmup_steps = 1
        self.adam_epsilon = 1e-8
        self.max_grad_norm = .3
        self.epochs = 100
        self.eval_steps = 2000
        self.seed = 42
        
args = Args()



In [5]:
class SudokuDataModule(pl.LightningDataModule):

  def __init__(self, X_train, X_test, y_train, y_test, args):
    super().__init__()
    
    self.X_train = X_train
    self.y_train = y_train
    self.X_test = X_test
    self.y_test = y_test
 
   

  def setup(self, stage=None):
    self.train_dataset = SudokuDataset(self.X_train,self.y_train)

    self.test_dataset = SudokuDataset(self.X_test,self.y_test)

  def train_dataloader(self):
    return DataLoader(
      self.train_dataset,
      batch_size=args.train_batch_size,
      shuffle=True,
      num_workers=2
    )

  def val_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=args.eval_batch_size,
      num_workers=2
    )

  def test_dataloader(self):
    return DataLoader(
      self.test_dataset,
      batch_size=self.eval_batch_size,
      num_workers=2
    )

In [6]:
data_module = SudokuDataModule(X_train, X_test, y_train, y_test, args)

In [7]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AdamW, get_linear_schedule_with_warmup
#import bitsandbytes as bnb

def _save_checkpoint(args, model, tokenizer,it):
    output_dir = os.path.join(args.output_dir, f"checkpoint_{it}")
    os.makedirs(output_dir, exist_ok=True)
    
    print(f"Saving model checkpoint to {output_dir}")

    model.save_pretrained(output_dir)
    tokenizer.save_pretrained(output_dir)
    torch.save(args, os.path.join(output_dir, "training_args.bin"))

def freeze(
    model,
    freeze_emb=True,
    freeze_ln=False,
    freeze_attn=False,
    freeze_ff=False,
    freeze_other=False,
):
    
    for name, p in model.named_parameters():
    # freeze all parameters except the layernorm and positional embeddings
       
       
        
        name = name.lower()
        if 'ln' in name or 'norm' in name:
            p.requires_grad = not freeze_ln
        elif 'embeddings' in name:
            p.requires_grad = not freeze_emb
        elif 'mlp' in name:
            p.requires_grad = not freeze_ff
        elif 'attn' in name:
            p.requires_grad = not freeze_attn
        else:
            p.requires_grad = not freeze_other
           
    return model

def collate(examples):
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.eos_token_id)





#optimizer =  bnb.optim.Adam8bit(model.parameters(), lr=args.lr,eps=args.adam_epsilon, betas=(0.9, 0.995))

#scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=args.lr, 
#                                                final_div_factor=500,  
#                                               steps_per_epoch=len(train_dataloader),
#                                               epochs=args.epochs 
#                                                )



In [8]:
import wandb

wandb.login(key='1d66b96a01b6d89816f35c71af8991f35ac96153')
wandb.init(project = 'sudoku', entity = "alexwortega")

/home/user/conda/lib/python3.7/site-packages/notebook/utils.py:282: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)
wandb: Currently logged in as: alexwortega (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publically.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jovyan/.netrc
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [9]:
class ToxicCommentTagger(pl.LightningModule):

  def __init__(self, args, n_training_steps=None, n_warmup_steps=None):
    super().__init__()
    self.gpt = GPTNeoForCausalLM.from_pretrained("EleutherAI/gpt-neo-2.7B")
    
    self.n_training_steps = n_training_steps
    self.n_warmup_steps = n_warmup_steps
    

  def forward(self, input_ids, attention_mask, labels):
    loss = self.gpt(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)[0]
   
   
    return loss

  def training_step(self, batch, batch_idx):
    input_ids, attention_mask, labels = batch
    
    loss = self.gpt(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)[0]
    self.log("train_loss", loss, prog_bar=True, logger=True)
    return {"loss": loss}

  def validation_step(self, batch, batch_idx):
    input_ids, attention_mask, labels = batch
    
    loss = self.gpt(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)[0]
    self.log("val_loss", loss, prog_bar=True, logger=True)
    return loss

  def test_step(self, batch, batch_idx):
    input_ids, attention_mask, labels = batch
    
    loss = self.gpt(input_ids=input_ids, labels=input_ids, attention_mask=attention_mask)[0]
    
    self.log("test_loss", loss, prog_bar=True, logger=True)
    return loss

  def training_epoch_end(self, outputs):
    
    pass

    #for i, name in enumerate(LABEL_COLUMNS):
    # class_roc_auc = auroc(predictions[:, i], labels[:, i])
    #  self.logger.experiment.add_scalar(f"{name}_roc_auc/Train", class_roc_auc, self.current_epoch)


  def configure_optimizers(self):

    optimizer = AdamW(self.parameters(), lr=2e-5)

    scheduler = get_linear_schedule_with_warmup(
      optimizer,
      num_warmup_steps=self.n_warmup_steps,
      num_training_steps=self.n_training_steps
    )

    return dict(
      optimizer=optimizer,
      lr_scheduler=dict(
        scheduler=scheduler,
        interval='step'
      )
    )

In [10]:
steps_per_epoch=len(X_train) // args.train_batch_size
total_training_steps = steps_per_epoch * args.epochs
warmup_steps = total_training_steps // 5

In [11]:
model = ToxicCommentTagger(
  args=args,
  n_warmup_steps=warmup_steps,
  n_training_steps=total_training_steps 

)

In [12]:
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
checkpoint_callback = ModelCheckpoint(
  dirpath="checkpoints",
  filename="best-checkpoint",
  save_top_k=1,
  verbose=True,
  monitor="val_loss",
  mode="min"
)
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=2)

In [13]:
from pytorch_lightning.loggers import WandbLogger

wandb_logger = WandbLogger(project="MNIST")

In [14]:
trainer = pl.Trainer(
  logger=wandb_logger,
  checkpoint_callback=checkpoint_callback,
  callbacks=[early_stopping_callback],
  max_epochs=2,
  gpus=1,
  devices=[1],
  progress_bar_refresh_rate=30
)

/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:440: UserWarning: The flag `devices=[1]` will be ignored, instead the device specific number 1 will be used
  f"The flag `devices={devices}` will be ignored, "
/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:152: LightningDeprecationWarning: Setting `Trainer(checkpoint_callback=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f2837d87950>)` is deprecated in v1.5 and will be removed in v1.7. Please consider using `Trainer(enable_checkpointing=<pytorch_lightning.callbacks.model_checkpoint.ModelCheckpoint object at 0x7f2837d87950>)`.
  f"Setting `Trainer(checkpoint_callback={checkpoint_callback})` is deprecated in v1.5 and will "
/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:97: LightningDeprecationWarning: Setting `Trainer(progress_bar_refresh_r

In [ ]:
trainer.fit(model, data_module)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]
/home/jovyan/.imgenv-gpt-neo-4gpu-0/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,

  | Name | Type              | Params
-------------------------------------------
0 | gpt  | GPTNeoForCausalLM | 2.7 B 
-------------------------------------------
2.7 B     Trainable params
0         Non-trainable params
2.7 B     Total params
10,605.230Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 256 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

/home/user/conda/lib/python3.7/site-packages/pytorch_lightning/loggers/wandb.py:346: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  "There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse"
